## Exploratory analysis

In [1]:
# #Optional: run analysis on subset of dataset
# search = 'dieren|natuur|natura|CO2|milieuklimaat|biodiversiteit'
# print(df['Text'].str.contains(search).sum())
# df = df.loc[df['Text'].str.contains(search)]
# df = (df.loc[df['Topic'].isin([23,27,21,20,8])])
# 1+1

In [32]:
import altair as alt
import numpy as np
import pandas as pd
import sklearn
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
import altair as alt
import pandas as pd
from collections import Counter
from collections import Counter
import pprint


In [39]:
import pickle
file = open("moties_processed_df.pickle","rb")
df = pickle.load(file)
print(len(df))

column_list = df.columns.values
indiener_column = [c for c in column_list if 'Indiener' in c]
medeindiener_column = [c for c in column_list if 'Medeindiener' in c]
stem_column = [c for c in column_list if 'Stem_' in c and c != 'Stem_persoon']
aantal_stemmen_column = [c for c in column_list if 'Aantal' in c]

29514


In [4]:
def get_largest_parties(year=2020,top=False):
    tmp = df[(df['Vergaderjaar']==str(year)+'-'+str(year+1))]
    parties = [p for p in tmp.columns if 'Aantal_stemmen' in p][1:]
    tmp = tmp[parties].mean().sort_values(ascending=False)
    tmp = tmp[tmp.notna()]
    tmp.index = tmp.index.str[15:]
    if top: return tmp[:top].index
    else: return tmp.index

def to_vergaderjaar(year):
    return str(year) + '-' + str(year+1)
    
get_largest_parties(2020)

Index(['VVD', 'PVV', 'D66', 'CDA', 'SP', 'GroenLinks', 'PvdA', 'ChristenUnie',
       'PvdD', 'SGP', 'DENK', 'FVD', 'Van Kooten-Arissen', 'Van Haga', 'Krol'],
      dtype='object')

In [5]:
from collections import namedtuple
import datetime
Cabinet = namedtuple('Cabinet', ['name','start', 'end', 'demissionair','coalition'])
Chamber = namedtuple('Chamber', ['name','start', 'end', 'numyears'])

chamber_input = [
    "Balkenende IV,01 01 2009,16 06 2010",# mark begin 2008 is not correct (should be 30 11 2006) but for dataset purpose it it
    "Rutte I,17 06 2010,19 09 2012",
    "Rutte II,20 09 2012,22 03 2017",
    "Rutte III,23 03 2017,31 12 2020"] # mark end 2020 is not correct but for dataset purpose it it

chambers = {}
for c in chamber_input:
    c = c.split(',')
    name = c[0]
    start = list(map(int, c[1].split()))
    startdate = datetime.datetime(start[2], start[1], start[0])
    end = list(map(int, c[2].split()))
    enddate = datetime.datetime(end[2], end[1], end[0])
    numyears = (enddate - startdate).days/365
    chambers[name] = Chamber(name, startdate, enddate, numyears)


cabinets_input = [
    # "Balkenende III,07 07 2006,22 02 2007,CDA VVD",
    # "Balkenende IV,22 02 2007,14 10 2010,CDA PvdA ChristenUnie",
    "Balkenende IV,18 09 2008,14 10 2010,20 02 2010,CDA PvdA ChristenUnie",
    "Rutte I,14 10 2010,5 11 2012,23 04 2012,VVD CDA PVV",
    "Rutte II,5 11 2012,26 10 2017,14 03 2017,VVD PvdA",
    "Rutte III,26 10 2017,15 01 2022,15 01 2022,CDA VVD D66 ChristenUnie"]

cabinets = {}
for c in cabinets_input:
    c = c.split(',')
    name = c[0]
    start = list(map(int, c[1].split()))
    startdate = datetime.datetime(start[2], start[1], start[0])
    end = list(map(int, c[2].split()))
    enddate = datetime.datetime(end[2], end[1], end[0])
    coalition = tuple(c[3].split())
    cabinets[name] = Cabinet(name, startdate, enddate, numyears, coalition)
cabinets, chambers

({'Balkenende IV': Cabinet(name='Balkenende IV', start=datetime.datetime(2008, 9, 18, 0, 0), end=datetime.datetime(2010, 10, 14, 0, 0), demissionair=3.778082191780822, coalition=('20', '02', '2010')),
  'Rutte I': Cabinet(name='Rutte I', start=datetime.datetime(2010, 10, 14, 0, 0), end=datetime.datetime(2012, 11, 5, 0, 0), demissionair=3.778082191780822, coalition=('23', '04', '2012')),
  'Rutte II': Cabinet(name='Rutte II', start=datetime.datetime(2012, 11, 5, 0, 0), end=datetime.datetime(2017, 10, 26, 0, 0), demissionair=3.778082191780822, coalition=('14', '03', '2017')),
  'Rutte III': Cabinet(name='Rutte III', start=datetime.datetime(2017, 10, 26, 0, 0), end=datetime.datetime(2022, 1, 15, 0, 0), demissionair=3.778082191780822, coalition=('15', '01', '2022'))},
 {'Balkenende IV': Chamber(name='Balkenende IV', start=datetime.datetime(2009, 1, 1, 0, 0), end=datetime.datetime(2010, 6, 16, 0, 0), numyears=1.4547945205479451),
  'Rutte I': Chamber(name='Rutte I', start=datetime.datetime(

In [6]:
df.columns

Index(['Titel', 'AgendapuntOnderwerp', 'Onderwerp', 'Datum', 'GestartOp',
       'Vergaderjaar', 'Jaar', 'Kamer', 'Kabinet', 'Demissionair',
       ...
       'Stem_PvdA', 'Stem_PvdD', 'Stem_SGP', 'Stem_SP', 'Stem_VVD',
       'Stem_Van Haga', 'Stem_Van Klaveren', 'Stem_Van Kooten-Arissen',
       'Stem_Van Vliet', 'Stem_Verdonk'],
      dtype='object', length=140)

In [7]:
for chamber in chambers:
    source_year = df.loc[df['Kamer'] == chamber][stem_column].dropna(axis=1, how='all').T
    X_year = SimpleImputer(strategy='most_frequent', fill_value=0).fit_transform(source_year)
    pca = PCA(n_components=1)
    pca = pca.fit(X_year)
    print('PCA results for',chamber)
    print(pca.explained_variance_ratio_,pca.explained_variance_ratio_.sum())  
res_year = pca.transform(X_year)
source = pd.DataFrame(res_year)
source['partij'] = source_year.T.columns.str[5:]
source = source.rename(index=str, columns={0: "x"}).sort_values('x',ascending=False)
# this variable will be used throughout the notebook!
parties = source['partij'][::-1]
parties = [p for p in parties if p in get_largest_parties(2020,12)]

alt.Chart(source,width=400,height = 150).mark_bar().encode(
    x=alt.X('partij:N',sort=alt.EncodingSortField(field="x", op="count", order='ascending')),
    y='x:Q'
)

PCA results for Balkenende IV
[0.33805194] 0.33805193516230103
PCA results for Rutte I
[0.40181041] 0.4018104099572369
PCA results for Rutte II
[0.26915201] 0.2691520148559245
PCA results for Rutte III
[0.33743251] 0.3374325093563288


alt.Chart(...)

## Ontwikkeling van het aantal moties

In [8]:
data = df.groupby(['Jaar', 'BesluitSoort']).agg(Aantal=('Titel', 'count')).reset_index()
data = data.replace({0: 'Afgewezen', 1: 'Aangenomen'})
color_order = {'Afgewezen':1, 'Aangenomen':0}
data["idx"] = data["BesluitSoort"].map(color_order)

alt.Chart(data, width=400,height = 150).mark_bar().encode(
    x=alt.X('Jaar:N'),
    y='Aantal:Q',
    order="idx",
    color=alt.Color('BesluitSoort',
                   scale=alt.Scale(
            domain=['Aangenomen','Afgewezen'],
            range=['green', 'red']))
)

alt.Chart(...)

We onderzoeken alle aangenomen moties tussen 2009 en 2020. In totaal is er over bijna 30.000 moties gestemd. Het aantal moties stijgt, en met name het aantal aangenomen moties. In verkiezingsjaren (2010, 2012 en 2017) ligt het aantal moties duidelijk lager. Tussen 2012 en 2013 werden relatief veel moties afgewezen.

In [9]:
data = df.groupby(['Jaar', 'BesluitSoort']).agg(Aantal=('Voor', 'mean')).reset_index()
data = data.replace({0: 'Afgewezen', 1: 'Aangenomen'})

# source = source.melt('index',var_name='Stem', value_name='Gemiddeld aantal voorstemmen')
# source['index'] += 2008
chart = alt.Chart(data,width=400,height = 150).mark_line().encode(
    x=alt.X('Jaar:N', axis=alt.Axis(title='Jaar')),
    y='Aantal:Q',
    color=alt.Color('BesluitSoort:N',
                scale=alt.Scale(
        domain=['Aangenomen','Afgewezen'],
        range=['green', 'red']))

)
chart.encoding.y.title = 'Aantal voorstemmen'
chart

alt.Chart(...)

Moties die worden aangenomen halen meestal rond de 120. Moties die het niet halen blijven meestal rond de 50 stemmen hangen. Eigenlijk zegt zo'n gemiddelde niet zoveel en kan je beter naar de verdeling van het aantal voorstemmen kijken, dat doen we nu met een histogram per kabinetsperiode.

In [10]:
df['Voor'].hist(by=df['Kamer'], bins=20, figsize=(7,7));

In bovenstaande histogrammen gaat het vooral om de verdeling van de moties. Het absolute aantal is niet echt vergelijkbaar omdat de duur van de kabinetten verschillend is. 
* Bij Balkenende IV zijn meerdere pieken te zien: protestmoties die nauwelijks stemmen krijgen, een grote piek met moties die rond de 35 stemmen blijven hangen en een piek met unaniem aangenomen moties
* Bij Rutte I waren er relatief veel meer moties die het nipt wel of juist niet haalden.
* Bij Rutte II is verlijkbaar met Balkenende IV, behalve dat er meer moties het niet haalden
* Bij Rutte III is er een hele grote piek in het aantal unaniem aangenomen moties. Dit lijken er ook steeds meer te worden (500 in 2018, 700 in 2019 en 900 in 2020)

In [11]:
medeindieners = df['Medeindiener_persoon'].values
c = Counter(len(i) for i in medeindieners)
c.most_common()
source = pd.DataFrame(c.most_common()).rename(columns={0:'Aantal medeindieners',1:'Aantal moties'})
alt.Chart(source,width=400,height = 150).mark_bar().encode(
    x=alt.X('Aantal medeindieners:O',sort=alt.EncodingSortField(field="x", op="count", order='ascending')),
    y='Aantal moties:Q'
)

alt.Chart(...)

Meestal worden moties ingediend door 1 partij, maar soms zijn er medeindieners. Sinds 2008 is er [één special snowflake motie](https://www.tweedekamer.nl/kamerstukken/detail?id=2020Z20755&did=2020D44509), met maar liefst 16 medeindieners. Van harte Dion Graus!

## Komt elke partij opdagen?

In [12]:
# how many percent of moties did parties vote. Conclusion almost all
nan_per_party = df[stem_column].isnull().groupby([df['Kamer']]).sum(axis=0)
count_per_chamber = df.groupby(['Kamer']).count()['Titel']
relevant_parties = ['Stem_'+p for p in get_largest_parties(2020)]
1- np.divide(nan_per_party, count_per_chamber.values[:,None])[relevant_parties]

,Stem_VVD,Stem_PVV,Stem_D66,Stem_CDA,Stem_SP,Stem_GroenLinks,Stem_PvdA,Stem_ChristenUnie,Stem_PvdD,Stem_SGP,Stem_DENK,Stem_FVD,Stem_Van Kooten-Arissen,Stem_Van Haga,Stem_Krol
Kamer,,,,,,,,,,,,,,,
Balkenende IV,1.000000,0.985759,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.999604,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000
Rutte I,0.999790,1.000000,1.000000,1.000000,1.000000,0.999790,0.999790,1.000000,1.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000
Rutte II,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000
Rutte III,0.999041,0.999041,0.999041,0.999041,0.999041,0.999041,0.999041,0.999041,0.998954,0.999041,0.99756,0.978824,0.340741,0.354771,0.147625


In [13]:
relevant_parties = ['Aantal_stemmen_'+p for p in get_largest_parties(2020)]
mean_votes_per_party = df.replace(np.nan, 0).groupby('Kamer').mean()[relevant_parties]
max_votes_per_party = df[(relevant_parties+['Kamer'])].groupby('Kamer').quantile(.95)
np.divide(mean_votes_per_party, max_votes_per_party.values)

,Aantal_stemmen_VVD,Aantal_stemmen_PVV,Aantal_stemmen_D66,Aantal_stemmen_CDA,Aantal_stemmen_SP,Aantal_stemmen_GroenLinks,Aantal_stemmen_PvdA,Aantal_stemmen_ChristenUnie,Aantal_stemmen_PvdD,Aantal_stemmen_SGP,Aantal_stemmen_DENK,Aantal_stemmen_FVD,Aantal_stemmen_Van Kooten-Arissen,Aantal_stemmen_Van Haga,Aantal_stemmen_Krol
Kamer,,,,,,,,,,,,,,,
Balkenende IV,0.998700,0.987342,1.003033,0.999344,0.999525,1.000170,0.999988,0.999802,0.999604,1.000198,NaN,NaN,NaN,NaN,NaN
Rutte I,0.999993,0.987544,1.000126,0.999760,1.000000,0.999412,0.999958,1.000000,1.000000,1.000315,NaN,NaN,NaN,NaN,NaN
Rutte II,0.986980,0.854401,0.999876,1.000000,1.000000,0.999605,0.966950,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN
Rutte III,0.986202,0.997050,0.999041,0.999041,0.999041,0.999041,0.999041,0.999041,0.912157,0.999041,0.99756,0.978824,0.340741,0.354771,0.147625


In [14]:
# there is something where a motion has been voted upon later, e.g. below motion is in Balkenende IV, but PVV only got to 24 chairs in Rutte I
df[(df['Kamer']=='Balkenende IV') & (df['Aantal_stemmen_PVV']==24)]

,Titel,AgendapuntOnderwerp,Onderwerp,Datum,GestartOp,Vergaderjaar,Jaar,Kamer,Kabinet,Demissionair,...,Stem_PvdA,Stem_PvdD,Stem_SGP,Stem_SP,Stem_VVD,Stem_Van Haga,Stem_Van Klaveren,Stem_Van Kooten-Arissen,Stem_Van Vliet,Stem_Verdonk
2010Z06296,Prenatale screening,Stemming over: aangehouden motie ingediend bij...,Motie-Langkamp/Arib over een EHBO-certificaat ...,2010-04-08,2010-04-08,2009-2010,2010,Balkenende IV,Balkenende IV,True,...,1.0,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN
2010Z06641,Voorstel van wet van de leden Koser Kaya en Bl...,Stemming over: motie ingediend bij Voorstel va...,Motie Omtzigt c.s. over modernisering van de g...,2010-04-13,2010-04-13,2009-2010,2010,Balkenende IV,Balkenende IV,True,...,1.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN


### Van Koorten en de Bie: Welke partij is de 'tegenpartij'?

In [15]:
startjaar = 2020
data = df.groupby(['Kamer']).mean()[['Stem_'+p for p in get_largest_parties(startjaar,12)]]
data = data.reset_index()
data = data.melt('Kamer',var_name='Stem', value_name='Stemgedrag')

alt.Chart(data,width=600).mark_line().encode(
    x='Kamer:N',
    y=alt.Y('Stemgedrag:Q'),
    color='Stem:N',
)

alt.Chart(...)

VVD (oranje) stemt het meeste tegen. Waarschijnlijk omdat de meeste moties gericht zijn aan de regering en de VVD eigenlijk altijd regeert

## Welke partijen dienen het meeste moties in en hoe succesvol zijn ze?

In [16]:
def get_summary_stats(df):
    df = df.dropna(axis='columns',how='all')
    indiener_column = [c for c in df.columns if 'Indiener' in c]
    indiener = {p:p[9:] for p in indiener_column}
    df = df.rename(indiener,axis=1)
    largest_parties = get_largest_parties(2020,12)
    indiener_column = [i for i in indiener.values() if i in largest_parties]
    aantal_stemmen_column = [c for c in df.columns if 'Aantal_stemmen' in c]
      
    success_rate = np.multiply(df[indiener_column],df['BesluitSoort'][:, np.newaxis]).mean(axis=0).sort_index()
    success_rate = success_rate[success_rate.notna()]
    aantal_moties = df[indiener_column].sum().sort_index()
    aantal_moties = aantal_moties[aantal_moties.notna()]
    aantal_succesvolle_moties = np.multiply(success_rate,aantal_moties)
    aantal_succesvolle_moties = aantal_succesvolle_moties[aantal_succesvolle_moties.notna()]
    aantal_niet_succesvolle_moties = np.multiply((1-success_rate),aantal_moties)
    aantal_niet_succesvolle_moties = aantal_niet_succesvolle_moties[aantal_niet_succesvolle_moties.notna()]
    aantal_zetels = df[aantal_stemmen_column].mean()
    aantal_zetels = aantal_zetels[['Aantal_stemmen_'+p for p in aantal_moties.index]]
    aantal_zetels.index = aantal_moties.index
    aantal_zetels = aantal_zetels[aantal_zetels.notna()]
    aantal_zetels.index = aantal_moties.index
    aantal_moties_zetel = np.divide(aantal_moties,aantal_zetels)
    aantal_succesvolle_moties_zetel = np.divide(aantal_succesvolle_moties,aantal_zetels)
    aantal_niet_succesvolle_moties_zetel = np.divide((aantal_moties-aantal_succesvolle_moties),aantal_zetels)
    return success_rate,aantal_moties,aantal_succesvolle_moties,aantal_niet_succesvolle_moties, aantal_zetels, aantal_moties_zetel,aantal_succesvolle_moties_zetel,aantal_niet_succesvolle_moties_zetel

party_perf = pd.DataFrame({})
res_columns = ['success_rate','aantal_moties','aantal_succesvolle_moties','aantal_niet_succesvolle_moties', 'aantal_zetels', 'aantal_moties_zetel','aantal_succesvolle_moties_zetel','aantal_niet_succesvolle_moties_zetel']
for k in df['Kamer'].unique():
    result = get_summary_stats(df.loc[df['Kamer'] == k])
    for i,res in enumerate(result):
        df_temp = pd.DataFrame(res).reset_index()
        df_temp['stat']=res_columns[i]
        df_temp['kamer']= k
        party_perf = party_perf.append(df_temp, ignore_index = True)
# for start_year in range(year,2020):
#     result = get_summary_stats(start_year,df.loc[df['Vergaderjaar'] == str(start_year)+'-'+str(start_year+1)])
#     for i,res in enumerate(result):
#         df_temp = pd.DataFrame(res).reset_index()
#         df_temp['stat']=res_columns[i]
#         df_temp['year']=str(start_year)+'-'+str(start_year+1)
#         party_perf = party_perf.append(df_temp, ignore_index = True)
party_perf = party_perf.rename({'index':'partij',0:'value'},axis=1)

In [17]:
# normalize metrics starting with 'aantal' to account for length of ca
mask = party_perf['stat'].str.contains('aantal')
valid = party_perf[mask]
party_perf.loc[mask, 'value'] = valid['value'] / valid['kamer'].map(lambda x: chambers[x].numyears)

In [62]:
# Step 1: create the line
metric_of_interest = 'aantal_succesvolle_moties_zetel'
data = party_perf[party_perf['stat'] == metric_of_interest]
data['partij'] = pd.Categorical(data['partij'], parties)
data = data.sort_values("partij")
# https://imagecolorpicker.com/
party_colors = {
'CDA':'#5cb957',
 'ChristenUnie':'#00a5e8',
 'D66':'#04a438',
 'GroenLinks':'#48a641',
 'PVV':'#002759',
 'PvdA':'#df111a',
 'PvdD':'#006b2d',
 'SGP':'orange',
 'SP':'#e3001b',
 'VVD':'#ff7f0e',
 'DENK':'#17becf',
 'FVD':'#800000'}
area = alt.Chart(party_perf
    #maybe dont do joinaggreage
    ).transform_joinaggregate(
    order='sum(value)',
    groupby=['partij']
    
    ).mark_area(interpolate="monotone").encode(
    x='kamer:O',
    y='value:Q',
    order=alt.Order('order:Q', sort='descending'),
    color=alt.Color('partij:N', scale=alt.Scale(
            domain=parties,
            range= [party_colors[p] for p in parties]))
).transform_filter(alt.FieldEqualPredicate(field='stat', equal=metric_of_interest))


def get_y_positions(party_perf, kamer):
    return data.groupby(["kamer", "partij"]).sum().loc[kamer].iloc[::-1].cumsum()["value"]

def get_bar_width(kamer, partij):
    query = f'kamer == "{kamer}" & partij == "{partij}"'
    return data.query(query)['value'].values[0]
    
xpos = ['Rutte III' if i%3!=2
                else 'Rutte II' if i%3==0
                else 'Rutte I' 
                for i, p in enumerate(parties)]

ys = {c: get_y_positions(party_perf, c) for c in chambers}
ypos = [ys[xpos[i]].loc[p] - 0.5* get_bar_width(xpos[i],p) for i, p in enumerate(parties)]
test = [get_bar_width(xpos[i],p) for i, p in enumerate(parties)]
demotext = pd.DataFrame({'kamer': xpos, 'value': ypos, 'partij': parties, 'test':test})
demotext

t = (alt.Chart(demotext).mark_text()
        .encode(
    x='kamer:O',
    y='value:Q',
    text='partij:N'
))

chart = alt.layer(area, t, width=600)
chart

alt.LayerChart(...)

Interessant! Bij veel partijen levert een zetel ongeveer 17 succesvolle moties op. Uitzonderingen zijn FvD en PvVV, maar ook Denk, PvdD, VVD en SP zijn minder productief. Opvallend dat de VVD eigenlijk al jaren lang weinig succesvolle moties indient, terwijl andere regeringspartijen dit wel doen.

In [25]:
party_perf = party_perf.loc[party_perf['partij'].isin(get_largest_parties(2020))]
chart = alt.Chart(party_perf).mark_bar().encode(
    column='kamer',
    x='value:Q',
    y='partij',
    color=alt.Color('stat',
                scale=alt.Scale(
        domain=['aantal_succesvolle_moties','aantal_niet_succesvolle_moties'],
        range=['green', 'red']))
).properties(width=100).transform_filter(alt.FieldOneOfPredicate(field='stat', oneOf=['aantal_succesvolle_moties','aantal_niet_succesvolle_moties']))
chart.encoding.x.title = 'Aantal moties'
chart

alt.Chart(...)

Over deze visualisatie ben ik erg tevreden. Het geeft in een oogopslag weer hoe succesvol partijen zijn geweest gedurende de jaren. Je ziet bijvoorbeeld de PVV, PvdD en SP als partijen met veel moties, maar weinig succesvolle. Hoewel de sommige partijen hiervan goede ideeen hebben, zijn ze in elk geval niet succesvol in ze door te voeren.

Bij de regeringspartijen is het opvallend hoe succesvol hun moties zijn. Waarschijnlijk omdat ze samen een meerderheid hebben en dus zo hun moties geaccepteerd krijgen.

In [26]:
chart = alt.Chart(party_perf).mark_bar().encode(
    column='kamer',
    x='value:Q',
    y='partij',
    color=alt.Color('stat',
                scale=alt.Scale(
        domain=['aantal_succesvolle_moties_zetel','aantal_niet_succesvolle_moties_zetel'],
        range=['green', 'red']))
).properties(width=100).transform_filter(alt.FieldOneOfPredicate(field='stat', oneOf=['aantal_succesvolle_moties_zetel','aantal_niet_succesvolle_moties_zetel']))
chart.encoding.x.title = 'Aantal moties'
chart

alt.Chart(...)

Wederom valt hier het lage aantal moties op bij VVD per zetel. Het is natuurlijk ook de partij met het hoogste aantal zetels, maar toch. Ook is de 'output' per zetel is vrij stabiel bij de meeste partijen, hoewel een partij die in de regering gaat meteen een stuk minder moties indient (bijvoorbeeld de PvdA)

## Wie zijn de motiekanonnen?

In [29]:
indieners = df[(df['Kamer'] == 'Rutte III') & (df['BesluitSoort'] == 1)]['Indiener_persoon'].values
c = Counter([i[0] for i in indieners if i])
print(c.most_common(10))

[('R.K. Dik-Faber', 154), ('J.A.M.J. van den Berg', 119), ('E.M. Westerveld', 114), ('J.M. Paternotte', 98), ('C.E. Ellemeet', 94), ('M. Amhaouch', 94), ('P.H. Omtzigt', 92), ('E.E.W. Bruins', 91), ('S.C. Kröger', 91), ('J.M. van Eijs', 89)]


In [28]:
medeindieners = df['Medeindiener_persoon'].values
c = Counter(x for i in medeindieners for x in i)
c.most_common(10)

[('R.K. Dik-Faber', 674),
 ('C.G. van der Staaij', 574),
 ('J.S. Voordewind', 563),
 ('L.G.J. Voortman', 488),
 ('L. van Tongeren', 445),
 ('S. van Veldhoven', 438),
 ('J.J. (Jasper) van Dijk', 401),
 ('E. Ouwehand', 362),
 ('E.E.W. Bruins', 347),
 ('J.F. Klaver', 329)]

In [33]:
for chamber in chambers:
    indieners = df[(df['Kamer'] == chamber)]['Indiener_persoon'].values
    c = Counter([i[0] for i in indieners if i])
    print(chamber)
    pprint.pprint(c.most_common(10))

Balkenende IV
[('E. Ouwehand', 82),
 ('H.P.J. van Gerven', 62),
 ('W. van Gent', 60),
 ('M.L. Thieme', 60),
 ('K. van Velzen', 57),
 ('E.G.M. Roemer', 53),
 ('H. Brinkman', 49),
 ('A. Pechtold', 48),
 ('S.R. Fritsma', 48),
 ('D.J.G. Graus', 46)]
Rutte I
[('E. Ouwehand', 188),
 ('H.P.J. van Gerven', 175),
 ('E.E. Wiegman-van Meppelen Scheppink', 143),
 ('S. van Veldhoven', 117),
 ('J.S. Voordewind', 110),
 ('A. Slob', 100),
 ('J.J. (Jasper) van Dijk', 91),
 ('R.M. Leijten', 89),
 ('L. Jacobi', 88),
 ('W. van Gent', 87)]
Rutte II
[('R.K. Dik-Faber', 364),
 ('L. van Tongeren', 343),
 ('E. Ouwehand', 313),
 ('S. van Veldhoven', 257),
 ('H.P.J. van Gerven', 252),
 ('L.G.J. Voortman', 243),
 ('D.J.G. Graus', 224),
 ('R.M. Leijten', 215),
 ('J.J. (Jasper) van Dijk', 215),
 ('P.H. Omtzigt', 212)]
Rutte III
[('L. van Raan', 325),
 ('E.M. Westerveld', 251),
 ('S.M. Beckerman', 243),
 ('F.P. Wassenberg', 225),
 ('E. Ouwehand', 210),
 ('S.C. Kröger', 200),
 ('W.J. Moorlag', 184),
 ('T. Kuzu', 182)

In [34]:
for chamber in chambers:
    indieners = df[(df['Kamer'] == chamber) & (df['BesluitSoort'] == 1)]['Indiener_persoon'].values
    c = Counter([i[0] for i in indieners if i])
    print(f'Meeste succesvolle moties, {chamber}')
    pprint.pprint(c.most_common(10))

Meeste succesvolle moties, Balkenende IV
[('E.A. Cramer', 29),
 ('E.E. Wiegman-van Meppelen Scheppink', 27),
 ('B. van der Ham', 25),
 ('J.J. Atsma', 24),
 ('J.S. Voordewind', 23),
 ('W. van Gent', 23),
 ('G.P.J. Koopmans', 22),
 ('C.A. Ortega-Martijn', 22),
 ('E.W. Anker', 21),
 ('M. Peters', 20)]
Meeste succesvolle moties, Rutte I
[('S. van Veldhoven', 59),
 ('G.P.J. Koopmans', 49),
 ('E.E. Wiegman-van Meppelen Scheppink', 48),
 ('A. Slob', 41),
 ('J.S. Voordewind', 40),
 ('S. de Rouwe', 38),
 ('L. Jacobi', 36),
 ('E. Ouwehand', 36),
 ('H.P.J. van Gerven', 35),
 ('C.G. van der Staaij', 34)]
Meeste succesvolle moties, Rutte II
[('R.K. Dik-Faber', 170),
 ('S. van Veldhoven', 156),
 ('L. van Tongeren', 109),
 ('V.A. Bergkamp', 91),
 ('L.G.J. Voortman', 80),
 ('S.P.R.A. van Weyenberg', 77),
 ('J.C. (Jan) Vos', 75),
 ('E. Ouwehand', 71),
 ('A.A. (Albert) de Vries', 70),
 ('E.M.A. Smaling', 70)]
Meeste succesvolle moties, Rutte III
[('R.K. Dik-Faber', 154),
 ('J.A.M.J. van den Berg', 119),

## Welke partijen werken samen?

In [46]:
medeindiener_perc = pd.DataFrame({})
for chamber in chambers:
    moties_in_year = df.loc[(df['Kamer'] == chamber)]
    moties_in_year = moties_in_year.dropna(axis=1, how='all')
    indieners = [c for c in moties_in_year.columns if 'Indiener' in c and c != 'Indiener_persoon']
    medeindieners = [c for c in moties_in_year.columns if 'Medeindiener' in c and c != 'Medeindiener_persoon']
    stem = ['Stem_'+c[13:] for c in medeindieners]
    assert len(stem) == len(medeindieners)
    for p in indieners:
        partij_moties =  moties_in_year.loc[(moties_in_year[p]==1)]
        res = (partij_moties[medeindieners].sum() / partij_moties[stem].notna().sum().values)
        res = pd.DataFrame({'support':res.index.str[13:],'percentage':res.values, 'mede':partij_moties[medeindieners].sum(), 'stem':partij_moties[stem].notna().sum().values})
        res['partij']= p[9:]
        res['kamer']= chamber
        # how often did the party cosign & vote / how often did the party vote
        res2 =partij_moties[medeindieners].values * partij_moties[stem].values
        res2 = pd.DataFrame({'support': [s[5:] for s in stem], 'percentage':np.nansum(res2, axis=0)/partij_moties[stem].notna().sum().values})
        res2['partij']= p[9:]
        res2['kamer']= chamber
        medeindiener_perc = medeindiener_perc.append(res2)      

In [47]:
# There are some cases where a party is submitting a moties but not voting on it, see below example
# https://www.tweedekamer.nl/kamerstukken/detail?id=2020Z22006&did=2020D46879

In [48]:
medeindiener_perc = medeindiener_perc[medeindiener_perc['support'].isin(parties)]
medeindiener_perc = medeindiener_perc[medeindiener_perc['partij'].isin(parties)]

alt.Chart(medeindiener_perc,height=250,width=250).mark_rect().encode(
    x=alt.X('support:N', sort=list(parties)),
    y=alt.Y('partij:N', sort=list(parties)),    
    color=alt.Color('percentage', scale=alt.Scale(scheme='greens'))
).transform_filter(alt.FieldOneOfPredicate(field='kamer', oneOf=['Rutte III']))##.properties(width=200)

alt.Chart(...)

Duidelijk is hier te zien dat de regeringspartijen en de oppositiepartijen vaak samen moties indienen. De grootste blokken zijn de CU met CDA als medeindiener, VVD met CDA als medeindiener. En waar bij de meeste partijen er 1 persoon is die de motie indient, staat er bij het FvD steenvast ook een mede-indiener van dezelfde partij op de motie. Vandaar dat het FvD-FvD vakje ook groen kleurt.

In [50]:
alt.Chart(medeindiener_perc,height=150,width=80).mark_rect().encode(
    column='partij:N',
    x=alt.X('kamer:N'),
    y=alt.Y('support:N', sort=list(parties)),    
    color=alt.Color('percentage', scale=alt.Scale(scheme='greens')))

alt.Chart(...)

Hier zijn weer heel mooi de effecten te zien van in de regering zitten. Het tijdelijke pact tussen PvdA en VVD, en de huidige steun van de regeringspartijen aan elkaar. Andere opvallende zaken:
- SGP diende vroeger vaak met CDA en CU moties in, dit is de afgelopen jaren een stuk minder geworden
- Groenlinks en PvdA zijn vaak medeindiener bij elkaars moties
- Denk, FvD en PVV hebben vrijwel nooit medeindieners
- Bij moties van de ChristenUnie zijn de afgelopen jaren de regeringspartijen vaak medeindiener

In [51]:
stem_perc = pd.DataFrame({})
for chamber in chambers:
    moties_in_year = df.loc[(df['Kamer'] == chamber)]
    moties_in_year = moties_in_year.dropna(axis=1, how='all')
    indieners = [c for c in moties_in_year.columns if 'Indiener' in c]
    stem = [c for c in moties_in_year.columns if 'Stem' in c][1:]
    
    for p in indieners:
        partij_moties =  moties_in_year.loc[(moties_in_year[p]==1)] 
        
        partij_moties = partij_moties.replace(-1,0)
        res = (partij_moties[stem].sum()/partij_moties[stem].notna().sum())
        res = pd.DataFrame({'support':res.index.str[5:],'percentage':res.values})
        res['partij']= p[9:]
        res['kamer'] = chamber
        stem_perc = stem_perc.append(res)

In [52]:
stem_perc = stem_perc[stem_perc['support'].isin(parties)]
stem_perc = stem_perc[stem_perc['partij'].isin(parties)]

alt.Chart(stem_perc,height=250,width=250).mark_rect().encode(
    column = 'kamer',
    x=alt.X('support:N', sort=list(parties)),
    y=alt.Y('partij:N', sort=list(parties)),
    color=alt.Color('percentage', scale=alt.Scale(scheme='redyellowgreen'))
).transform_filter(alt.FieldOneOfPredicate(field='kamer', oneOf=['Rutte III']))#.properties(width=200)

alt.Chart(...)

In [53]:
test = stem_perc[stem_perc['support']==stem_perc['partij']]

Au! Dat doet pijn bij FvD! Bijna alle partijen stemmen tegen hun moties, behalve PVV en 50Plus.
Van de oppositiepartijen krijgen alleen SGP, 50Plus, PvdA en Groenlinks regelmatig steun van de regeringspartijen. De andere partijen kunnen dit over het algemeen vergeten.
PvdD krijgt nog wat hulp van Groenlinks en de SP.

In [56]:
stem_perc = stem_perc[stem_perc['support'].isin(parties)]
stem_perc = stem_perc[stem_perc['partij'].isin(parties)]

alt.Chart(stem_perc,height=150,width=80).mark_rect().encode(
    column = 'partij',
    x=alt.X('kamer:N', sort=list(parties)),
    y=alt.Y('support:N', sort=list(parties)),
    color=alt.Color('percentage', scale=alt.Scale(scheme='redyellowgreen'))
)#.properties(width=200)

alt.Chart(...)

## Welke partijen lijken het meeste op elkaar qua stemgedrag?

In [57]:
for chamber in chambers:
    source_year = df.loc[df['Kamer'] == chamber][stem_column].dropna(axis=1, how='all').T
    X_year = SimpleImputer(strategy='most_frequent', fill_value=0).fit_transform(source_year)
    pca = PCA(n_components=1)
    pca = pca.fit(X_year)
    print('PCA results for', chamber)
    print(pca.explained_variance_ratio_,pca.explained_variance_ratio_.sum())  
res_year = pca.transform(X_year)
source = pd.DataFrame(res_year)
source['partij'] = source_year.T.columns.str[5:]
source = source.rename(index=str, columns={0: "x"}).sort_values('x',ascending=False)
# this variable will be used throughout the notebook!
parties = source['partij'][::-1]
parties = [p for p in parties if p in get_largest_parties(2020,12)]

alt.Chart(source,width=400,height = 150).mark_bar().encode(
    x=alt.X('partij:N',sort=alt.EncodingSortField(field="x", op="count", order='ascending')),
    y='x:Q'
)

PCA results for Balkenende IV
[0.33805194] 0.33805193516230325
PCA results for Rutte I
[0.40181041] 0.40181040995723644
PCA results for Rutte II
[0.26915201] 0.2691520148559257
PCA results for Rutte III
[0.33743251] 0.337432509356328


alt.Chart(...)

Als we het stemgedrag visualiseren op 1 dimensie is dit het overzicht. Deze dimensie verklaart 40% van het stemgedrag. Kennelijk liggen VVD en PvdD het meest van elkaar verwijderd.

In [58]:
for chamber in chambers:
    source_year = df.loc[df['Kamer'] == chamber][stem_column].dropna(axis=1, how='all').T
    X_year = SimpleImputer(strategy='most_frequent', fill_value=0).fit_transform(source_year)
    pca = PCA(n_components=2)
    pca = pca.fit(X_year)
    print('PCA results for', chamber,'number of parties',len(source_year))
    print('explained variance two first factors', pca.explained_variance_ratio_,pca.explained_variance_ratio_.sum())  
res_year = pca.transform(X_year)
source = pd.DataFrame(res_year)
source['label'] = source_year.T.columns.str[5:]
source = source.rename(index=str, columns={0: "x", 1: "y"})                      

size= 1600
points = alt.Chart(source,width= size * pca.explained_variance_ratio_[0],height = size * pca.explained_variance_ratio_[1]).mark_point().encode(
    x=alt.X('x:Q', axis=alt.Axis(title='Eerste factor')),
    y=alt.Y('y:Q', axis=alt.Axis(title='Tweede factor')),
    tooltip=['label']
)
text = points.mark_text(
    align='left',
    baseline='middle',
    dx=np.random.uniform(0,10),
    dy=np.random.uniform(0,10),
    opacity=0.5
).encode(
    text='label'
).properties(
    title='Welke partijen lijken op elkaar'
)
points + text

PCA results for Balkenende IV number of parties 12
explained variance two first factors [0.33805194 0.24502353] 0.5830754654843995
PCA results for Rutte I number of parties 16
explained variance two first factors [0.40181041 0.20534438] 0.6071547878303065
PCA results for Rutte II number of parties 21
explained variance two first factors [0.26915201 0.15211891] 0.4212709220627926
PCA results for Rutte III number of parties 17
explained variance two first factors [0.33743251 0.19181778] 0.5292502914825199


alt.LayerChart(...)

Er zijn 3 clusters te onderscheiden: de oppositie linksonder, de regeringspartijen rechtsonder en dan nog PVV en FvD bovenaan. De horizontale as is groter omdat deze meer van het stemgedrag verklaart. Uiteindelijk kan je met deze twee factoren al ~60% van het stemgedrag verklaren.

## Kunnen we de stemuitslag voorspellen?

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
start_year = 2008

scores = []
for s in range(start_year,2019):
    X = df.loc[df['Vergaderjaar'] == str(s)+'-'+str(s+1)][indiener_column+medeindiener_column].dropna(how='all',axis=1)
    X_imp = SimpleImputer(strategy='constant', fill_value=0).fit_transform(X)
    y = df.loc[df['Vergaderjaar'] == str(s)+'-'+str(s+1)]['BesluitSoort']
    X_train, X_test, y_train, y_test = train_test_split(X_imp, y, test_size=0.5, shuffle=False)


    clf = LogisticRegression(random_state=0, solver='lbfgs',
                             multi_class='ovr').fit(X_train, y_train)

    print('score',clf.score(X_test, y_test),'baseline',y.value_counts()[0]/len(y))
    scores.append(clf.score(X_test, y_test))
    print (confusion_matrix(y_test, clf.predict(X_test), sample_weight=None))
print (np.mean(scores))